# Labeling posts with Llama3-8b via Groq

We want to figure out what model would be good for our inference task. We've been experimenting with Llama3-8b via Groq and it seems to perform pretty well for our zero-shot task! Let's see how this performs on our pilot data.

We'll use Groq for this, and run it on 300 posts from our pilot data. We'll use the same prompts that we did from our demo Streamlit app.

We'll use the LiteLLM [Groq](https://litellm.vercel.app/docs/providers/groq) connection to connect to Groq.


In [1]:
import os

import pandas as pd

from ml_tooling.llm.inference import run_query
from ml_tooling.llm.prompt_helper import generate_complete_prompt_for_post_link

In [2]:
MODEL_NAME = "Llama3-8b (via Groq)"
current_wd = os.getcwd()
PILOT_DATA_FP = "../manuscript_pilot/representative_diversification_feed.csv"

In [3]:
pilot_data = pd.read_csv(PILOT_DATA_FP)

In [4]:
bluesky_post_links: list[str] = pilot_data["link"].tolist()

In [6]:
def create_prompts_for_each_link(links: list[str], task_name: str) -> list[str]:
    """Creates prompts for each link."""
    res = []
    for link in links:
        try:
            prompt = generate_complete_prompt_for_post_link(link, task_name)
            res.append(prompt)
        except Exception as e:
            print(f"Error with link {link}: {e}")
            continue
    return res

We'll classify civic and political lean in one go. I didn't find any difference between doing it this way vs chaining them (doing civic first and then doing political).

In [ ]:
# note: lots of request to Bsky to hydrate post and author info, so needed to
# remove the logs from the output. Takes ~4 minutes to run.
civic_and_political_lean_prompts = create_prompts_for_each_link(
    bluesky_post_links, "both"
)

Now we can export these prompts as a .jsonl

In [9]:
posts_to_classify: list[dict] = [
    {
        "link": link,
        "prompt": prompt,
        "task_name": "Civic and Political Lean"
    }
    for (link, prompt) in zip(bluesky_post_links, civic_and_political_lean_prompts)
]

In [11]:
jsonl_filename = "posts_to_classify.jsonl"
jsonl_fp = os.path.join(current_wd, jsonl_filename)

with open(jsonl_fp, "w") as f:
    for post in posts_to_classify:
        f.write(f"{str(post)}\n")

Now let's load the .jsonl to verify that it works

In [12]:
with open(jsonl_fp, 'r') as f:
    lines = f.readlines()

In [14]:
jsons_to_classify: list[dict] = [eval(line) for line in lines]

Now that we know that this works, let's start classifying. Let's first start by doing it on the first prompt.

In [16]:
example_post = jsons_to_classify[0]
example_prompt = example_post["prompt"]

In [19]:
example_result = run_query(
    prompt=example_prompt, model_name=MODEL_NAME
)

20:30:42 - LiteLLM:INFO: utils.py:1112 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.groq.com/openai/v1/ \
-d '{'model': 'llama3-8b-8192', 'messages': [{'role': 'user', 'content': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing 

In [20]:
example_result

'Here is the classification result in JSON format:\n\n{\n    "civic": "civic",\n    "political_ideology": "left-leaning",\n    "reason_civic": "The post references a specific policy proposal by Bernie Sanders, a left-leaning politician, and discusses the benefits of a four-day workweek.",\n    "reason_political_ideology": "The post\'s language and tone, as well as its reference to a left-leaning politician\'s proposal, suggest a left-leaning political ideology."\n}'

We want something that has a nicer printing. Ideally it also only returns the JSON format.

In [21]:
print(example_result)

Here is the classification result in JSON format:

{
    "civic": "civic",
    "political_ideology": "left-leaning",
    "reason_civic": "The post references a specific policy proposal by Bernie Sanders, a left-leaning politician, and discusses the benefits of a four-day workweek.",
    "reason_political_ideology": "The post's language and tone, as well as its reference to a left-leaning politician's proposal, suggest a left-leaning political ideology."
}


Let's try the next result

In [22]:
second_prompt = jsons_to_classify[1]["prompt"]

In [23]:
second_result = run_query(
    prompt=second_prompt, model_name=MODEL_NAME
)

20:32:13 - LiteLLM:INFO: utils.py:1112 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.groq.com/openai/v1/ \
-d '{'model': 'llama3-8b-8192', 'messages': [{'role': 'user', 'content': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing 

In [24]:
second_result

'Here is the classification result in JSON format:\n\n{\n    "civic": "civic",\n    "political_ideology": "right-leaning",\n    "reason_civic": "The post discusses social issues such as fear of kidnapping/CPS and loss of third spaces for teens, which are civic topics.",\n    "reason_political_ideology": "The post\'s emphasis on individual responsibility and parental protectionism is characteristic of right-leaning ideology."\n}\n\nNote that I classified the post as "civic" because it discusses social issues that affect a large group of people, such as fear of kidnapping/CPS and loss of third spaces for teens. I classified the post as "right-leaning" because it emphasizes individual responsibility and parental protectionism, which are characteristic of right-leaning ideology.'

Let's try to prompt the model to return only JSON format

In [27]:
second_prompt = (
    second_prompt + "\nReturn ONLY the JSON. I will parse the string result in JSON format."
)

In [28]:
second_result = run_query(
    prompt=second_prompt, model_name=MODEL_NAME
)

20:33:35 - LiteLLM:INFO: utils.py:1112 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.groq.com/openai/v1/ \
-d '{'model': 'llama3-8b-8192', 'messages': [{'role': 'user', 'content': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing 

In [29]:
print(second_result)

{
    "civic": "civic",
    "political_ideology": "right-leaning",
    "reason_civic": "The post discusses social issues affecting teenagers, such as the decline of third spaces and the impact of fear on their social behavior.",
    "reason_political_ideology": "The post's emphasis on individual responsibility and the need for parental supervision suggests a right-leaning perspective."
}


In [31]:
second_result_dict: dict = eval(second_result)

In [32]:
second_result_dict

{'civic': 'civic',
 'political_ideology': 'right-leaning',
 'reason_civic': 'The post discusses social issues affecting teenagers, such as the decline of third spaces and the impact of fear on their social behavior.',
 'reason_political_ideology': "The post's emphasis on individual responsibility and the need for parental supervision suggests a right-leaning perspective."}

Let's go back and add this string command to all the prompts

In [33]:
for post in jsons_to_classify:
    post["prompt"] = post["prompt"] + "\nReturn ONLY the JSON. I will parse the string result in JSON format."

In [35]:
print(jsons_to_classify[0]["prompt"])



Pretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". 

Then, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are 'left-leaning', 'moderate', 'right-leaning', or 'unclear'. You are analyzing text that has been pre-identified as 'political' in nature. If the text is not civic, return "unclear".

Think through your response step by step.

Return in a JSON format in the following way:
{
    "civic": <

Now let's export these

In [36]:
jsonl_filename = "posts_to_classify.jsonl"
jsonl_fp = os.path.join(current_wd, jsonl_filename)

with open(jsonl_fp, "w") as f:
    for post in jsons_to_classify:
        f.write(f"{str(post)}\n")

Now let's rerun our inference

In [37]:
example_post = jsons_to_classify[100]
example_prompt = example_post["prompt"]
example_result = run_query(
    prompt=example_prompt, model_name=MODEL_NAME
)

20:38:03 - LiteLLM:INFO: utils.py:1112 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.groq.com/openai/v1/ \
-d '{'model': 'llama3-8b-8192', 'messages': [{'role': 'user', 'content': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing 

In [39]:
example_res: dict = eval(example_result)

In [40]:
example_res

{'civic': 'civic',
 'political_ideology': 'left-leaning',
 'reason_civic': "The post references a news article about Elon Musk's charity and tax benefits, indicating a civic concern about government policies and philanthropy.",
 'reason_political_ideology': "The post's tone and language, such as 'it would be cool if laws existed', suggests a left-leaning perspective critical of wealth inequality and corporate influence."}

Great! Let's run this at scale now. Let's also save our intermediate results to see how they do.

In [44]:
inference_results: list[str] = []
total_num_posts = len(jsons_to_classify)

In [ ]:
# note: this inferacts with the Groq API directly, so this will cost money.
# rate limit is 30 requests/min (https://console.groq.com/settings/limits)
# in total, this took ~16 minutes.
for idx, post in enumerate(jsons_to_classify):
    if idx % 50 == 0:
        print(f"Processing post {idx + 1} of {total_num_posts}")
    try:
        post_result = run_query(
            prompt=post["prompt"], model_name=MODEL_NAME
        )
        inference_results.append(post_result)
    except Exception as e:
        print(f"Error with post {idx + 1}: {e}")
        break

Let's see how it did

In [47]:
res = eval(inference_results[0])

In [48]:
res

{'civic': 'civic',
 'political_ideology': 'left-leaning',
 'reason_civic': 'The post references a specific policy proposal by a politician (Bernie Sanders) and discusses its potential benefits.',
 'reason_political_ideology': "The post's tone and language, as well as its reference to a left-leaning politician's proposal, suggest a left-leaning political ideology."}

In [49]:
classified_results: list[dict] = [
    {
        **post, "result": result
    }
    for (post, result) in zip(jsons_to_classify, inference_results)
]

In [50]:
classified_results[0]

{'link': 'https://bsky.app/profile/jbouie.bsky.social/post/3knqbtrdzrz2n',
 'prompt': '\n\nPretend that you are a classifier that predicts whether a post has civic content or not. Civic refers to whether a given post is related to politics (government, elections, politicians, activism, etc.) or social issues (major issues that affect a large group of people, such as the economy, inequality, racism, education, immigration, human rights, the environment, etc.). We refer to any content that is classified as being either of these two categories as “civic”; otherwise they are not civic. Please classify the following text denoted in <text> as "civic" or "not civic". \n\nThen, if the post is civic, classify the text based on the political lean of the opinion or argument it presents. Your options are \'left-leaning\', \'moderate\', \'right-leaning\', or \'unclear\'. You are analyzing text that has been pre-identified as \'political\' in nature. If the text is not civic, return "unclear".\n\nTh

Let's hydrate the results from the classification

In [73]:
hydrated_classified_results: list[dict] = []

In [74]:
for idx, post in enumerate(classified_results):
    hydrated_res = {**post}
    try:
        post_dict = eval(post["result"])
        hydrated_res["valid_json_response"] = True
        hydrated_res["hydrated_result"] = post_dict
        hydrated_res["civic_label"] = post_dict["civic"]
        hydrated_res["political_label"] = post_dict["political_ideology"]
        hydrated_res["reason_civic_label"] = post_dict["reason_civic"]
        hydrated_res["reason_political_label"] = post_dict["reason_political_ideology"]
    except Exception as e:
        # sometimes the LLM returns an invalid JSON response
        print(f"Error with post {post['link']} at index {idx}: {e}")
        hydrated_res["valid_json_response"] = False
        hydrated_res["hydrated_result"] = None
        hydrated_res["civic_label"] = None
        hydrated_res["political_label"] = None
        hydrated_res["reason_civic_label"] = None
        hydrated_res["reason_political_label"] = None
    finally:
        hydrated_classified_results.append(hydrated_res)

Error with post https://bsky.app/profile/blueheronfarm.bsky.social/post/3knmci6lfht2e at index 31: '{' was never closed (<string>, line 1)
Error with post https://bsky.app/profile/brendelbored.bsky.social/post/3knobbmewi22h at index 59: '{' was never closed (<string>, line 1)
Error with post https://bsky.app/profile/mangmangmang.bsky.social/post/3kno26hkn222u at index 72: name 'null' is not defined
Error with post https://bsky.app/profile/alexanderchee.bsky.social/post/3knincv5iv225 at index 88: name 'null' is not defined
Error with post https://bsky.app/profile/kjhealy.bsky.social/post/3knl7425rb22z at index 93: '{' was never closed (<string>, line 1)
Error with post https://bsky.app/profile/maxkennerly.bsky.social/post/3kndtrgi6ql25 at index 95: name 'null' is not defined
Error with post https://bsky.app/profile/jbouie.bsky.social/post/3knstcm42kk2h at index 128: '{' was never closed (<string>, line 1)
Error with post https://bsky.app/profile/golikehellmachine.bsky.social/post/3knh3a

Now let's export our results.

In [82]:
classified_posts_filename = "classified_posts.jsonl"
classified_posts_fp = os.path.join(current_wd, classified_posts_filename)

with open(classified_posts_fp, "w") as f:
    for post in hydrated_classified_results:
        f.write(f"{str(post)}\n")

In [83]:
classified_posts_df: pd.DataFrame = pd.DataFrame(hydrated_classified_results)

Now let's see how many valid JSONs there were

In [80]:
num_valid_jsons: int = sum(classified_posts_df["valid_json_response"])

In [81]:
print(f"Number of valid JSON responses: {num_valid_jsons}")
print(f"Total number of posts: {len(classified_posts_df)}")
print(f"Proportion of valid JSON responses: {num_valid_jsons / len(classified_posts_df)}")

Number of valid JSON responses: 335
Total number of posts: 354
Proportion of valid JSON responses: 0.9463276836158192


Now let's compare results to the GPT labels

In [77]:
pilot_data_cols = ["link", "civic", "political_ideology"]

In [78]:
joined_df: pd.DataFrame = pd.merge(
    pilot_data[pilot_data_cols],
    classified_posts_df, left_on="link", right_on="link"
)

In [79]:
joined_df.head()

,link,civic,political_ideology,prompt,task_name,result,hydrated_result,civic_label,political_label,reason_civic_label,reason_political_label,valid_json_response
0,https://bsky.app/profile/jbouie.bsky.social/po...,True,left-leaning,\n\nPretend that you are a classifier that pre...,Civic and Political Lean,"{\n ""civic"": ""civic"",\n ""political_ideol...","{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,The post references a specific policy proposal...,"The post's tone and language, as well as its r...",True
1,https://bsky.app/profile/lethalityjane.bsky.so...,True,right-leaning,\n\nPretend that you are a classifier that pre...,Civic and Political Lean,"{\n ""civic"": ""civic"",\n ""political_ideol...","{'civic': 'civic', 'political_ideology': 'righ...",civic,right-leaning,The post discusses social issues affecting tee...,The post's emphasis on individual responsibili...,True
2,https://bsky.app/profile/esqueer.bsky.social/p...,True,left-leaning,\n\nPretend that you are a classifier that pre...,Civic and Political Lean,"{\n ""civic"": ""civic"",\n ""political_ideol...","{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,"The post discusses censorship and neo-nazism, ...",The post's criticism of neo-nazism and its ass...,True
3,https://bsky.app/profile/stuflemingnz.bsky.soc...,False,NaN,\n\nPretend that you are a classifier that pre...,Civic and Political Lean,"{\n ""civic"": ""not civic"",\n ""political_i...","{'civic': 'not civic', 'political_ideology': '...",not civic,unclear,,,True
4,https://bsky.app/profile/sararoseg.bsky.social...,True,left-leaning,\n\nPretend that you are a classifier that pre...,Civic and Political Lean,"{\n ""civic"": ""civic"",\n ""political_ideol...","{'civic': 'civic', 'political_ideology': 'left...",civic,left-leaning,The post discusses historical events related t...,The post presents a critical view of the Nazi ...,True


Let's only get the ones that we got valid results from 

In [85]:
filtered_results_df: pd.DataFrame = joined_df[joined_df["valid_json_response"]]

Now, given these, let's compare the labels from GPT4 to the labels from Llama3-8b

In [87]:
# from GPT4
print(filtered_results_df["civic"].value_counts())
print(filtered_results_df["political_ideology"].value_counts())

civic
False    173
True     162
Name: count, dtype: int64
political_ideology
left-leaning     129
unclear           16
right-leaning     15
 left-leaning      1
moderate           1
Name: count, dtype: int64


In [86]:
# from Llama3-8b
print(filtered_results_df["civic_label"].value_counts())
print(filtered_results_df["political_label"].value_counts())

civic_label
civic        189
not civic    146
Name: count, dtype: int64
political_label
unclear          175
left-leaning     123
right-leaning     34
moderate           3
Name: count, dtype: int64


Let's fix some of the labels from the pilot data. There is a " left-leaning" value that should be "left-leaning"

In [91]:
# in the "political_ideology" column, there is a " left-leaning" value that needs
# to be replaced with "left-leaning"
filtered_results_df["political_ideology"] = filtered_results_df["political_ideology"].replace(" left-leaning", "left-leaning")

/var/folders/9g/nhh_jmv17sd2604y43zdctzw0000gn/T/ipykernel_23716/3734042687.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_results_df["political_ideology"] = filtered_results_df["political_ideology"].replace(" left-leaning", "left-leaning")


In [92]:
print(filtered_results_df["political_ideology"].value_counts())

political_ideology
left-leaning     130
unclear           16
right-leaning     15
moderate           1
Name: count, dtype: int64


OK, now let's compare civic-ness between both models using a confusion matrix

In [93]:
# create an cross-tab of the "civic" and "civic-label" columns. Rename the 
# "civic" as "GPT-4 label" and "civic-label" as "Llama3-8b label". Where
# the "GPT4-label" is True, replace with "civic" (not in place, only in the crosstab)
# and replace "False" with "not civic"
civic_crosstab: pd.DataFrame = pd.crosstab(
    filtered_results_df["civic"],
    filtered_results_df["civic_label"].apply(lambda x: x == "civic")
)

In [97]:
gpt_4_civic_results = filtered_results_df["civic"]
gpt_4_civic_results = gpt_4_civic_results.replace(True, "civic")
gpt_4_civic_results = gpt_4_civic_results.replace(False, "not civic")
llama_3_8b_civic_results = filtered_results_df["civic_label"]

In [109]:
# compare gpt4 to llama results. Name each axis based on the model that it
# came from ("GPT-4" or "Llama3-8b")
civic_crosstab: pd.DataFrame = pd.crosstab(
    gpt_4_civic_results, llama_3_8b_civic_results,
    colnames=["Llama3-8b (columns)"], rownames=["GPT-4 (rows)"],
    margins=True
)


In [110]:
# columns are Llama3-8b, rows are GPT-4.
# there are a total of 189 civic posts from Llama (162 from GPT4)
civic_crosstab

Llama3-8b (columns),civic,not civic,All
GPT-4 (rows),,,
civic,106,56,162
not civic,83,90,173
All,189,146,335


Now let's calculate the cross-tab as a proportion

In [107]:
civic_crosstab_props: pd.DataFrame = pd.crosstab(
    gpt_4_civic_results, llama_3_8b_civic_results,
    colnames=["Llama3-8b (columns)"], rownames=["GPT-4 (rows)"],
    margins=True, normalize="all"
)

In [108]:
civic_crosstab_props

Llama3-8b,civic,not civic,All
GPT-4,,,
civic,0.316418,0.167164,0.483582
not civic,0.247761,0.268657,0.516418
All,0.564179,0.435821,1.000000


Now let's calculate precision, recall, and F1 score based on the confusion matrix

In [111]:
from sklearn.metrics import precision_recall_fscore_support

In [115]:
y_true = gpt_4_civic_results.tolist()
y_pred = llama_3_8b_civic_results.tolist()

In [117]:
y_true[0:5]

['civic', 'civic', 'civic', 'not civic', 'civic']

In [118]:
civic_metrics = precision_recall_fscore_support(
    y_true=y_true, y_pred=y_pred, average="binary", pos_label="civic"
)

In [120]:
civic_precision, civic_recall, civic_fbeta_score, civic_support = civic_metrics

In [123]:
print(f"Precision: {civic_precision}\tRecall: {civic_recall}\tF-1 score: {civic_fbeta_score}\tSupport: {civic_support}")

Precision: 0.5608465608465608	Recall: 0.654320987654321	F-1 score: 0.603988603988604	Support: None


Let's now do the same comparison for political ideology. For the political ideology, we only want posts that are civic. Let's take the posts that both GPT4 and Llama3-8b agreed are civic.

In [126]:
# both "civic_label" == "civic" and "civic" == True
civic_posts: pd.DataFrame = filtered_results_df[
    (filtered_results_df["civic_label"] == "civic")
    & (filtered_results_df["civic"] == True)
]

In [127]:
civic_posts = civic_posts[["political_ideology", "political_label"]].rename(
    columns={"political_ideology": "GPT-4 political ideology", "political_label": "Llama3-8b political ideology"}
)

In [128]:
civic_posts.head()

,GPT-4 political ideology,Llama3-8b political ideology
0,left-leaning,left-leaning
1,right-leaning,right-leaning
2,left-leaning,left-leaning
4,left-leaning,left-leaning
5,unclear,left-leaning


In [130]:
print(civic_posts["GPT-4 political ideology"].value_counts())
print(civic_posts["Llama3-8b political ideology"].value_counts())

GPT-4 political ideology
left-leaning     83
right-leaning    11
unclear          11
moderate          1
Name: count, dtype: int64
Llama3-8b political ideology
left-leaning     70
right-leaning    20
unclear          14
moderate          2
Name: count, dtype: int64


In [133]:
political_ideology_confusion_matrix = pd.crosstab(
    civic_posts["GPT-4 political ideology"], civic_posts["Llama3-8b political ideology"],
    rownames=["GPT-4 political ideology (rows)"], colnames=["Llama3-8b political ideology (columns)"],
    margins=True
)

In [134]:
political_ideology_confusion_matrix

Llama3-8b political ideology (columns),left-leaning,moderate,right-leaning,unclear,All
GPT-4 political ideology (rows),,,,,
left-leaning,62,0,12,9,83
moderate,0,0,1,0,1
right-leaning,2,2,5,2,11
unclear,6,0,2,3,11
All,70,2,20,14,106


In [135]:
political_ideology_confusion_matrix_props = pd.crosstab(
    civic_posts["GPT-4 political ideology"], civic_posts["Llama3-8b political ideology"],
    rownames=["GPT-4 political ideology (rows)"], colnames=["Llama3-8b political ideology (columns)"],
    margins=True, normalize="all"
)

In [136]:
political_ideology_confusion_matrix_props

Llama3-8b political ideology (columns),left-leaning,moderate,right-leaning,unclear,All
GPT-4 political ideology (rows),,,,,
left-leaning,0.584906,0.000000,0.113208,0.084906,0.783019
moderate,0.000000,0.000000,0.009434,0.000000,0.009434
right-leaning,0.018868,0.018868,0.047170,0.018868,0.103774
unclear,0.056604,0.000000,0.018868,0.028302,0.103774
All,0.660377,0.018868,0.188679,0.132075,1.000000


In [137]:
political_ideology_metrics = precision_recall_fscore_support(
    y_true=civic_posts["GPT-4 political ideology"].tolist(),
    y_pred=civic_posts["Llama3-8b political ideology"].tolist(),
    average="weighted"
)

In [138]:
political_ideology_recall, political_ideology_precision, political_ideology_fbeta_score, political_ideology_support = political_ideology_metrics

In [139]:
print(f"Precision: {political_ideology_precision}\tRecall: {political_ideology_recall}\tF-1 score: {political_ideology_fbeta_score}\tSupport: {political_ideology_support}")

Precision: 0.660377358490566	Recall: 0.7417115902964959	F-1 score: 0.6929845372922957	Support: None
